Домашнее задание №1
====



In [68]:
# Импортируем нужные нам библиотеки
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import numpy as np
import matplotlib.dates as mdates
import seaborn as sns
import matplotlib
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(15, 10)})
sns.set(font_scale=2)
sns.set_style("whitegrid")
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import TimeSeriesSplit
import math


In [2]:
# Загружаем датасет, с которым мы будем работать
df = pd.read_csv('../datatest/bike-sharing.csv')

In [3]:
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,9.84,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,9.02,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,9.02,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,9.84,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,9.84,0.2879,0.75,0.0,0,1,1


Задание
====
Для тех же данных, что мы рассмотрели на лекции (bike sharing), сделать следующее:
* Выбрать самостоятельно уровень агрегации (день, неделя, месяц, год), а так же функцию агрегации (среднее, сумма, максимум). 
* Построить базовую наивную модель
* В этот раз применить принцип кросс-валидации для временных рядов, используя выбранную метрику ($R^2$, MSE, MAE, MRSE).




In [8]:
df['dteday'] = pd.to_datetime(df['dteday'])
df.set_index('dteday', inplace=True)

In [24]:
df = df.resample('W').mean()
df.sort_values('dteday')

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,cnt_hat
dteday,,,,,,,,,,,,,,,,,
2011-01-02,26.0,1.000000,0.0,1.0,12.581395,0.000000,2.790698,0.000000,1.837209,14.969767,0.366088,0.748605,0.222491,10.302326,28.883721,39.186047,39.302326
2011-01-09,128.5,1.000000,0.0,1.0,11.820988,0.000000,3.012346,0.703704,1.259259,7.754568,0.192397,0.493333,0.212834,4.123457,53.950617,58.074074,58.086420
2011-01-16,291.0,1.000000,0.0,1.0,11.742331,0.000000,3.000000,0.705521,1.349693,7.535951,0.189537,0.534479,0.203652,4.134969,51.233129,55.368098,55.294479
2011-01-23,449.0,1.000000,0.0,1.0,12.117647,0.156863,3.078431,0.542484,1.555556,7.412157,0.182322,0.545294,0.222324,3.954248,53.366013,57.320261,57.300654
2011-01-30,595.0,1.000000,0.0,1.0,11.956835,0.000000,2.892086,0.669065,1.625899,7.846043,0.207873,0.680863,0.144749,4.474820,50.913669,55.388489,55.388489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-09,16769.5,4.000000,1.0,12.0,11.500000,0.000000,3.000000,0.714286,1.595238,15.867976,0.387526,0.725119,0.163654,22.142857,202.630952,224.773810,224.773810
2012-12-16,16937.5,4.000000,1.0,12.0,11.500000,0.000000,3.000000,0.714286,1.583333,13.769167,0.338282,0.668274,0.165962,18.333333,195.750000,214.083333,214.190476
2012-12-23,17105.5,2.714286,1.0,12.0,11.500000,0.000000,3.000000,0.714286,1.476190,13.500714,0.326562,0.625714,0.221576,12.654762,146.250000,158.904762,158.952381


In [46]:
df['cnt_hat'] = df['cnt'].shift(1)
df.dropna(inplace=True)
X = df["cnt"]
y = df["cnt_hat"]

dteday
2011-02-13     62.260606
2011-02-20     68.662577
2011-02-27     89.042424
2011-03-06     76.664557
2011-03-13     71.163636
                 ...    
2012-12-09    210.401198
2012-12-16    224.773810
2012-12-23    214.083333
2012-12-30    158.904762
2013-01-06     64.578313
Freq: W-SUN, Name: cnt_hat, Length: 100, dtype: float64

In [72]:
tscv = TimeSeriesSplit(n_splits=5)
i = 0
print("TRAIN:", '\t MSE\t MAE\t RMSe \tR2')
for train_index, test_index in tscv.split(X):
    i += 1
    X_train, X_test = X[train_index], X[test_index]
    #print(len(X_train))
    y_train, y_test = y[train_index], y[test_index]
    mse = round(mean_squared_error(X_train, y_train), 2)
    mae = round(mean_absolute_error(X_train, y_train), 2)
    rmse = round(math.sqrt(mse), 2)
    r2 = round(r2_score(X_train, y_train), 2)
    print(i, '\t\t', mse, mae, rmse, r2)

TRAIN: 	 MSE	 MAE	 RMSe 	R2
1 		 225.81 11.0 15.03 0.91
2 		 406.56 14.27 20.16 0.8
3 		 524.06 17.12 22.89 0.7
4 		 538.11 17.48 23.2 0.8
5 		 524.59 17.44 22.9 0.88
